## 7.函数装饰器和闭包
# 7.1装饰器基础知识
1. 装饰器是可调用的对象，参数是另一个函数（被装饰的函数）。
```
@decorate
def fun()
等同于fun = decorate(fun)
```
2. 装饰器可以处理被装饰函数后再返回，也可以直接替换成另一个函数或可调用的对象（此时被装饰函数名为返回函数的引用）

In [1]:
def deco(func):
    def inner():
        print('inner()')
    return inner
@deco
def target():
    print('target')

target()
print(target)# target为inner函数的引用

inner()
<function deco.<locals>.inner at 0x7fb9f82171f0>


## 7.2 Python何时执行装饰器
1. 装饰器在被装饰的函数定义之后立即运行。若作为模块，为导入模块时就运行

In [3]:
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
main()#装饰器函数先运行

running register(<function f1 at 0x7fb9f8217280>)
running register(<function f2 at 0x7fb9f8217160>)
running main()
registry -> [<function f1 at 0x7fb9f8217280>, <function f2 at 0x7fb9f8217160>]
running f1()
running f2()
running f3()


## 7.4变量作用域规则
1. 因为函数体对b进行赋值了，Python编译函数定义体时，判断b为局部变量，Python会尝试从本地环境中获取b， 则会报错，因为局部变量b还没有绑定任何值（在全局变量与局部变量同名的情况下并有赋值行为，未声明是全局变量，函数体默认为局部变量，无论局部变量赋值语句出现的先后顺序）

In [8]:
b = 1
def f(a):
    print(a)
    print(b)#局部变量b还没有绑定任何值
    b=2
f(3)


3


UnboundLocalError: local variable 'b' referenced before assignment

In [14]:
b=1
def f2(a):
    global b#把b设为全局变量
    print(a)
    print(b)
    b=2
f2(3)
print(b)

3
1
2


In [15]:
from dis import dis
dis(f)
#LOAD_FAST                1 (b) 先加载局部变量

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (2)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7.5闭包
1. 闭包含义：在一个内部函数中，对外部作用域的变量进行引用，(并且一般外部函数的返回值为内部函数)，那么内部函数就被认为是闭包。Python考虑到内函数还会引用局部变量，就将局部变量保存到自由变量中。有面向对象的特性

In [23]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_averager()
print(avg)#avg 为内部函数averag的引用
print(avg(10))
print(avg(20))
print(avg.__code__.co_varnames)#new_value total 保存在局部变量中
print(avg.__code__.co_freevars)#series变量没有释放，而是保存在自由变量中
print(avg.__closure__[0].cell_contents)#自由变量的值保存在函数__closure__中

<function make_averager.<locals>.averager at 0x7fba002e0b80>
10.0
15.0
('new_value', 'total')
('series',)
[10, 20]


## 7.5 nonlocal声明
1. 若在内函数中重新绑定外部函数的局部变量（赋值操作），会导致该变量无法成为自由变量，不会保存在闭包中（参照7.4变量作用域）
## 7.6
1. nonlocal可以将变量标记为自由变量，即便重新赋值也会变成自由变量（类比gobal）

In [24]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count,total#若不声明则会提示没有count、total变量，因为前面已经释放了
        count += 1
        total += new_value
        return total/count
    return averager
avg = make_averager()
print(avg(10))
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)#count、total 保存在自由变量中
print(avg.__closure__[0].cell_contents)#自由变量的值保存在函数__closure__中

10.0
('new_value',)
('count', 'total')
1


## 7.7 实现函数计时装饰器
1. functools.wraps 可以将被装饰的函数的_\_name__ _\_doc__等属性传递给装饰器的闭包函数中，并能处理关键字参数

In [47]:
import time
def clock(func):
    def clocked(*args,**kwargs):
        """计算函数运行时间 """
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        pari = ['%s=%r' %(k,w) for k,w in sorted(kwargs.items())]#处理关键参数
        print(','.join(pari))
        return result
    return clocked

@clock
def fun_a(n,keyt=2):
    """ 打印参数 """
    print(n)
fun_a(1)
print(fun_a)
print(fun_a.__name__)
print(fun_a.__doc__)#fun_a函数的属性被clocked覆盖

1
[0.00004625s] fun_a(1) -> None
<function clock.<locals>.clocked at 0x7fba002f8310>
clocked
计算函数运行时间 
None


In [58]:
import time,functools
def clock(func):
    @functools.wraps(func)#functools.wraps 装饰器把相关的属性从 func 复制到 clocked 中。
    def clocked(*args,**kwargs):
        """计算函数运行时间 """
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        pari = ['%s=%r' %(k,w) for k,w in sorted(kwargs.items())]
        print(','.join(pari))
        return result
    return clocked

@clock
def fun_b(n,keyt=2):
    """ 打印参数 """
    print(n)
fun_b(1,keyt=10)
print(fun_b)
print(fun_b.__name__)
print(fun_b.__doc__)#属性均未变

1
[0.00021195s] fun_b(1) -> None
keyt=10
<function fun_b at 0x7fba002f8ca0>
fun_b
 打印参数 


## 7.8标准库中的装饰器
1. functools.lru_rache，可以实现备忘录功能，将函数结果缓存起来，避免重复计算，缓存不会无限制增长，一段时间不用的缓存条目就会被丢掉
    + functools.1ru_cache(maxsize=128, typed=False)，
    + maxsize——存储多少个调用结果，
    + typed 是否将不同参数类型得到的结果分开保存。如1 和 1.0
    + lru_cache使用字典存储结果，键值根据传入的定位参数和关键字参数创建，要求lru_cache装饰的函数，参数是可散列的（不变类型）

In [65]:
@clock
def fibonacci(n):
    if n<2: 
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(4))

[0.00000000s] fibonacci(0) -> 0

[0.00000000s] fibonacci(1) -> 1

[0.00011063s] fibonacci(2) -> 1

[0.00000000s] fibonacci(1) -> 1

[0.00000119s] fibonacci(0) -> 0

[0.00000095s] fibonacci(1) -> 1

[0.00068212s] fibonacci(2) -> 1

[0.00075006s] fibonacci(3) -> 2

[0.00095010s] fibonacci(4) -> 3

3


In [64]:
import functools
@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2: 
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(4))#重复参数的函数调用不会再计算结果

[0.00000000s] fibonacci(0) -> 0

[0.00000095s] fibonacci(1) -> 1

[0.00030804s] fibonacci(2) -> 1

[0.00000119s] fibonacci(3) -> 2

[0.00039291s] fibonacci(4) -> 3

3


2. [functools.singledispath](https://peps.python.org/pep-0443)——单分派泛函数。类比重载方法和函数，可接受不同类型参数，处理方式也随之不同

In [3]:
import functools
import numbers
from collections import abc
@functools.singledispatch
def base_fun(obj):
    print("This is a objection")
@base_fun.register(str)
def _(text):
    print("This is string %s" %text)

@base_fun.register(abc.MutableSequence)#可接收两种参数
@base_fun.register(numbers.Integral)
def _(n):
    print("THis is int or list")

base_fun([1])

THis is int or list


## 7.9叠放装饰器
1. 即多个装饰器可按顺序应用到一个函数上
    ```
    @d1
    @d2
    def fun():
        pass
    ```
    等同于fun = d1(d2(fun))

## 7.10参数化装饰器
1. 装饰器如何接收除被装饰函数以外的其他参数？————创建装饰器工厂函数，传参给装饰器（闭包原理，参数存在自由变量中）

In [36]:
def decorate_factory(flag=True):
    def decorate(fun):
        if flag:
            print(fun.__name__)
        return fun
    return decorate
@decorate_factory()
def fun_a():
    print("fun_a")
@decorate_factory(False)
def fun_b():
    print("fun_b")

fun_a


In [35]:
temp = decorate_factory(True)
temp(fun_b)
print(temp.__code__.co_freevars)#装饰器工厂函数的flag被放在自由变量中让参数器使用
print(temp)

fun_b
('flag',)


## 参考文档

wrapt/README.md at develop · GrahamDumpleton/wrapt
https://github.com/GrahamDumpleton/wrapt/blob/develop/blog/README.md

wrapt/01-how-you-implemented-your-python-decorator-is-wrong.md at develop · GrahamDumpleton/wrapt
https://github.com/GrahamDumpleton/wrapt/blob/develop/blog/01-how-you-implemented-your-python-decorator-is-wrong.md

wrapt — wrapt 1.13.0rc2 documentation
https://wrapt.readthedocs.io/en/latest/

decorator · PyPI
https://pypi.org/project/decorator/

PythonDecoratorLibrary - Python Wiki
https://wiki.python.org/moin/PythonDecoratorLibrary

